#### **Imports**

In [ ]:
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from Helper_Functions import *
from tqdm.notebook import tqdm
import time
from scipy.optimize import linear_sum_assignment

import joblib
from scipy.ndimage import gaussian_filter


#### **Load Pre-Trained ARMAX models**

In [ ]:
model_x = joblib.load('Model_x.pkl')
model_y = joblib.load('Model_y.pkl')
results_x = model_x.fit()
results_y = model_y.fit()

#### **Other Functions**

In [ ]:
# Returns positions and ensures they are in the correct orientation

def Filter_Positions(datum,edata,event_keys,team_1,tkeep_pos):
    
    locs,teams,keeper,actors = Get_Positions(datum)
    ev = Get_Associated_Event(datum,edata,event_keys)
    if  team_1 == ev['team']['name']:

        tlocs = locs[(teams)&(~keeper)]
        olocs = locs[(~teams)&(~keeper)]
        tkeep = locs[(teams)&(keeper)]
        okeep = locs[(~teams)&(keeper)]
        bp = locs[actors]

    else:
        locs[:,0] = 120-locs[:,0]
        locs[:,1] = 80-locs[:,1]
        tlocs = locs[(~teams)&(~keeper)]

        olocs = locs[(teams)&(~keeper)]
        tkeep = locs[(~teams)&(keeper)]
        okeep = locs[(teams)&(keeper)]
        bp = locs[actors]


    actor_loc = locs[actors]
    if team_1 == ev['team']['name']:
        expected_location = ev['location']
    else:
        expected_location = np.array([120,80]) - ev['location']
    

    alternative_location = np.array([120,80]) - expected_location

    if np.sum(np.abs(actor_loc - expected_location)) > np.sum(np.abs(actor_loc - alternative_location))  + 5:
        #print('here')
        tlocs[:,0] = 120-tlocs[:,0]
        tlocs[:,1] = 80-tlocs[:,1]

        olocs[:,0] = 120-olocs[:,0]
        olocs[:,1] = 80-olocs[:,1]

        tkeep[:,0] = 120-tkeep[:,0]
        tkeep[:,1] = 80-tkeep[:,1]

        okeep[:,0] = 120-okeep[:,0]
        okeep[:,1] = 80-okeep[:,1]  
        
        bp[:,0] = 120-bp[:,0]
        bp[:,1] = 80 - bp[:,1]
    if len(tkeep) > 0:
        if tkeep_pos == 'low' and tkeep[0,0] > 80:
            tkeep = []
        if tkeep_pos == 'high' and tkeep[0,0] < 40:
            tkeep = []
    if len(okeep) > 0:
        if tkeep_pos == 'low' and okeep[0,0] < 40:
            okeep = []
        if tkeep_pos == 'high' and okeep[0,0] > 80:
            okeep = []
    t = 60*ev['minute'] + ev['second'] - 45*60*(ev['period'] - 1)
    
    return tlocs,tkeep,olocs,okeep,bp,t

###########################################################################
# Log-likelihood distance function

def Ndist(x,mu,sigma):

    return np.sum(-(np.square(x-mu))/(2*(sigma**2)) - np.log((sigma**2)))

##############################################################################
# Mean predictions for positions using ARMAX model

def Predict(all_positions,all_times,curr_frame,all_exog):
    if len(all_positions) > 100:
        all_positions = all_positions[-100:]
        all_times = all_times[-100:]
        curr = 100
        all_exog = all_exog[curr_frame-100:]
    else:
        curr = curr_frame
    models = [results_x,results_y]
    outputs = np.zeros((20,2,2))
    for m in range(20):
        for k in range(2):
            p = all_positions[:curr,m,k]
            t = all_times[:curr,m]
            exog = all_exog[:curr,k]
            p_smoothed = np.zeros(curr)
            t[t==-10] = 0
            discretes = np.arange(len(t)).astype(int)
            t = np.append(-50,t)
            p_filt= p[t[1:] != t[:-1]]
            
            
            
            t_filt = t[1:][t[1:] != t[:-1]]
            d_filt =discretes[t[1:] != t[:-1]]
            if len(p) > 1:
                index = np.max(np.arange(len(p_smoothed))[t[1:] != t[:-1]])
                
                
                if curr - index > 1:
                
                    for n in range(len(p_filt)-1):
                        p_smoothed[int(d_filt[n]):int(d_filt[n+1])] = np.linspace(p_filt[n],p_filt[n+1],int(d_filt[n+1] - d_filt[n]))
                    p_smoothed[int(d_filt[len(p_filt)-1])] = p_filt[len(p_filt)-1]
                    if len(p_smoothed) > 1 and index > 1:
                        try:
                            pred_results = models[k].apply(p_smoothed[:index+1],exog=exog[:index+1])

                            preds = pred_results.get_prediction(start=index, end=curr,exog = all_exog[index+1:curr+1,k])

                            outputs[m,k,0] = preds.predicted_mean[-1]
                            outputs[m,k,1] = preds.se_mean[-1]
                        except:
                            outputs[m,k,0] = p[-1]
                            outputs[m,k,1] = (curr-index)*4
          
                    else:
                        outputs[m,k,0] = p[-1]
                        outputs[m,k,1] = (curr-index)*4
                else:
                    outputs[m,k,0] = p[-1]
                    outputs[m,k,1] = (curr-index)*4
            else:

                outputs[m,k,0] = p[-1]
                outputs[m,k,1] = (curr_frame+1)*4
    return outputs


#############################################
# Main processing loop
def Process(direc,period):
    initialise = True
    curr_positions = np.zeros((22,2))
    curr_times = (-10)*np.ones(22)

    all_positions = np.zeros((15000,22,2))
    all_times = np.zeros((15000,22))
    true_times = np.zeros((15000,22))
    visible = np.zeros((15000,22))
    all_exog = np.zeros((15000,2))
    try:
        pdata,edata,event_keys = Load_Data(direc)
    except:
        print(direc)

    team_1 = Get_Associated_Event(pdata[0],edata,event_keys)['team']['name']
    bs = []
    ts = []
    for datum in pdata:
        ev = Get_Associated_Event(datum,edata,event_keys)
        if ev['period'] == period:
            tlocs,tkeep,olocs,okeep,b_pos,t = Filter_Positions(datum,edata,event_keys,team_1,'No')

        else:
            continue
        if len(ts) > 0:
            if t == ts[-1]:
                continue
        if b_pos.shape == (1,2):
            bs.append(b_pos)
        else:
            if len(b_pos) > 0:
                bs.append([b_pos[0,:]])
            else:
                bs.append(bs[-1])
        ts.append(t)
    ts = np.array(ts)
    bs = np.array(bs)

    xs = np.interp(np.arange(int(np.max(ts))),ts,bs[:,:,0].reshape(len(bs)))
    ys = np.interp(np.arange(int(np.max(ts))),ts,bs[:,:,1].reshape(len(bs)))
    all_exog = np.zeros((len(xs),2))
    all_exog[:,0] = xs
    all_exog[:,1] = ys

    all_exog = gaussian_filter(all_exog,sigma=4)
    dat_num = 0
    tkeep_pos = 'No'
    for datum in tqdm(pdata):
        ev = Get_Associated_Event(datum,edata,event_keys)
        if ev['period'] == period:
            tlocs,tkeep,olocs,okeep,b_pos,t = Filter_Positions(datum,edata,event_keys,team_1,tkeep_pos)
            if tkeep != []:
                if tkeep.shape == (2,2) or tkeep.shape == (3,2):
                    tkeep = []
            if okeep != []:
                if okeep.shape == (2,2) or okeep.shape == (3,2):
                    okeep = []
            if b_pos.shape == (1,2):
                b_pos = [b_pos[0,:]]
        else:
            continue
        if initialise:
            for n,loc in enumerate(tlocs):
                curr_positions[n] = loc
                curr_times[n] = 0
                visible[dat_num,n] = 1
            other_ys = np.linspace(0,80,(10-n+2))
            xloc = np.mean(curr_positions[:n,0])
            if xloc > 60:
                newx = 90
            else:
                newx = 30

            for m in range(n+1,10):
                curr_positions[m] =[newx,other_ys[m-n+1]]
                curr_times[m] = - 10

            if len(tkeep) > 0:
                curr_positions[20] = tkeep
                visible[dat_num,20] = 1
            else:
                if xloc > 60:
                    curr_positions[20] = [110,40]
                else:
                    curr_positions[20] = [10,40]
            if xloc > 60:
                tkeep_pos = 'high'
            else:
                tkeep_pos = 'low'

            for n,loc in enumerate(olocs):
                curr_positions[n+10] = loc
                curr_times[n+10] =  0
                visible[dat_num,n+10] = 1
            other_ys = np.linspace(0,80,(10-n+2))
            xloc = np.mean(curr_positions[10:10+n,0])
            if xloc > 60:
                newx = 90
            else:
                newx = 30

            for m in range(n+1,10):
                curr_positions[m+10] =[newx,other_ys[m-n+1]]
                curr_times[m+10] =  - 10



            if len(okeep) > 0:
                curr_positions[21] = okeep
                visible[dat_num,21] = 1
            else:
                if xloc > 60:
                    curr_positions[21] = [110,40]
                else:
                    curr_positions[21] = [10,40]


            initialise = False
            all_positions[dat_num] = curr_positions
            all_times[dat_num] = curr_times
            true_times[dat_num] = 0
            dat_num += 1


            continue




        outputs = Predict(all_positions[:dat_num],all_times[:dat_num],dat_num,all_exog)
        dists = np.zeros((10,len(tlocs)))
        for n in range(10):
            for m in range(len(tlocs)):
                dists[n,m] = Ndist(tlocs[m],outputs[n,:,0],outputs[n,:,1])
        row_indices, col_indices = linear_sum_assignment(-dists)
        for i, j in zip(row_indices, col_indices):
            curr_positions[i] = tlocs[j]
            curr_times[i] = t
            visible[dat_num,i] = 1


        dists = np.zeros((10,len(olocs)))
        for n in range(10):
            for m in range(len(olocs)):
                dists[n,m] = Ndist(olocs[m],outputs[n+10,:,0],outputs[n+10,:,1])
        row_indices, col_indices = linear_sum_assignment(-dists)
        for i, j in zip(row_indices, col_indices):
            curr_positions[i+10] = olocs[j]
            curr_times[i+10] = t
            visible[dat_num,i+10] = 1

        if len(tkeep) > 0:


            curr_positions[20] =tkeep
            curr_times[20] = t
            visible[dat_num,20] = 1

        if len(okeep) > 0:

            curr_positions[21] = okeep
            curr_times[21] = t
            visible[dat_num,21] = 1




        all_positions[dat_num] = curr_positions

        all_times[dat_num] = curr_times

        true_times[dat_num] = t
        dat_num+=1


    all_positions = all_positions[:dat_num]
    all_times = all_times[:dat_num]
    visible = visible[:dat_num]
    true_times = true_times[:dat_num]
    all_exog = all_exog[:dat_num] 

    mu = 0.6
    velocities = np.zeros((dat_num-1,2))
    for n in range(dat_num-1):
        allowed = (visible[n] == 1)&(visible[n+1] == 1)&(np.arange(22)<20)
        if np.sum(allowed) > 0:
            velocities[n] = np.mean(all_positions[n+1,allowed] - all_positions[n,allowed],0)*mu

    filtered_positions = np.copy(all_positions)
    for n in range(22):
        player_vis = visible[:,n] == 1
        vis_indices = np.arange(dat_num)[player_vis]
        vis_indices = np.append(vis_indices,dat_num-1)
        for old,new in zip(vis_indices[:-1],vis_indices[1:]):

            tstart = all_times[old,n]
            tend = all_times[new,n]

            pstart = all_positions[old,n]
            pend = all_positions[new,n]
            if tend == tstart:
                continue
            if n < 20:
                tot_veloc = np.sum(velocities[old:new-1],0)
                veloc_shifts = np.cumsum(velocities[old:new-1],0)
                veloc_shifts = np.append([[0,0]],veloc_shifts,axis=0)



                filtered_positions[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]-tot_veloc[0]) + pstart[0] + veloc_shifts[:,0]
                filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]-tot_veloc[1]) + pstart[1] + veloc_shifts[:,1]
                #filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]
            else:
                filtered_positions[old:new,n,0] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[0]-pstart[0]) + pstart[0]
                filtered_positions[old:new,n,1] = ((true_times[old:new,n]-tstart)/(tend-tstart))*(pend[1]-pstart[1]) + pstart[1]

    return filtered_positions,visible

###########################################
#Writing JSON files
def Write(direc,filtered_positions,visible):
    pdata,edata,event_keys = Load_Data(direc)
    
    dout = []
    for n,datum in enumerate(pdata):
        if n ==len(filtered_positions):
            break
        ff = datum['freeze_frame']
        for m in range(len(ff)):
            ff[m]['visible'] = True

        locs,teams,keeper,actors = Get_Positions(datum)
        aloc = locs[actors]
        if len(aloc) == 0:
            aloc = np.array([[-100,-100]])
        try:
            index = np.where(np.sum(np.abs(filtered_positions[n]- aloc),1) <1e-4)[0][0]
            shift=False
        except:
            try:
                index = np.where(np.sum(np.abs(filtered_positions[n] - np.array([120-aloc[0,0],80-aloc[0,1]])),1)<1e-4)[0][0]
                shift = True
            except:
                xpos = np.median(filtered_positions[n,:,0])
                if np.abs(xpos - aloc[0,0]) < np.abs(120-xpos-aloc[0,0]):
                    shift = False
                else:
                    shift = True
                if np.sum(actors&keeper) > 0:
                    index= np.argmin(np.abs(filtered_positions[n,20:,0] - aloc[0,0])) + 20
                else:
                    index = -1
                    new['status'] = 'error'

        if index < 10 or index == 20:
            teammate= True
            opp = False
        else:
            teammate = False
            opp = True
        for m in range(22):
            if visible[n,m] == 0:
                new = {}
                if m < 10 or m ==20:
                    new['teammate'] = teammate

                else:
                    new['teammate'] = opp

                new['actor'] = False
                if m < 20:
                    new['keeper'] = False
                else:
                    new['keeper'] = True
                if shift:
                    new['location'] = [120-filtered_positions[n,m,0],80-filtered_positions[n,m,1]]
                else:
                    new['location'] = list(filtered_positions[n,m])
                new['visible'] = False
                ff.append(new)

        new = {}
        new['event_uuid'] = datum['event_uuid']
        new['visible_area'] = datum['visible_area']
        new['freeze_frame'] = ff
        dout.append(new)

    dout = json.dumps(dout)
    with open('Outputs/'  + direc,'w') as f:
        f.write(dout)

#### **Processing**

In [ ]:
try:
    os.mkdir('Outputs')
except:
    continue


direcs = os.listdir('open-data/data/three-sixty')



for direc in tqdm(direcs):
    try:
        if os.path.isfile('Outputs/'  + direc):
            print(direc + ' done')
            continue

        filtered_positions_1,v_1 = Process(direc,1)
        filtered_positions_2,v_2 = Process(direc,2)
        fp = np.append(filtered_positions_1,filtered_positions_2,0)
        v=np.append(v_1,v_2,0)
        pdata,edata,event_keys = Load_Data(direc)
        if len(pdata) > len(fp):
            filtered_positions_3,v_3 = Process(direc,3)
            filtered_positions_4,v_4 = Process(direc,4)

            fp = np.append(fp,filtered_positions_3,0)
            fp = np.append(fp,filtered_positions_4,0)

            v=np.append(v,v_3,0)
            v=np.append(v,v_4,0)

        Write(direc,fp,v)
    except:
        if direc == '.ipynb_checkpoints':
            continue
        print(direc + ' failed')
        break

    
    
           